In [14]:
from datetime import datetime
from pathlib import Path
import configparser

import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import AnonymizedNumber, VALID_SOURCE_GEOGRAPHIC_RESOLUTIONS

### Define Dataset Parameters

In [2]:
RAW_DATA_DIR_PATH = Path('.') / 'raw_data'
ALIGNMENT_DATA_DIR_PATH =  Path('.') / 'alignment_data'

### Load Helper Data

This includes a mapping of categorial values to integers (to save space when saving data later).

In [ ]:
fips_state_code_dataframe = pd.read_csv(ALIGNMENT_DATA_DIR_PATH / 'fips_state_codes.csv')

numeric_mappings = configparser.ConfigParser()
numeric_mappings.optionxform = str
numeric_mappings.read('./numeric_mappings.ini')
numeric_mappings = {k: int(v) for k, v in dict(numeric_mappings['mapping']).items()}
numeric_mappings[np.nan] = -1

### Load LearnPlatform Dataset

In [5]:
(Path('.') / 'processed_data').mkdir(parents=False, exist_ok=True)

learnplatform_data_path = (RAW_DATA_DIR_PATH / 'LearnPlatform').resolve()
        
districts = pd.read_csv(learnplatform_data_path / 'districts_info.csv')
products  = pd.read_csv(learnplatform_data_path / 'products_info.csv')

engagement = {}
for district_engagement_path in (learnplatform_data_path / 'engagement_data').iterdir():
    engagement[int(district_engagement_path.stem)] = pd.read_csv(district_engagement_path)


### Filter LearnPlatform Dataset

In [6]:
## All rows have an anonymized locale if and only if they also have an anonymized
## state, which unfortunately means they aren't useful for any kind of prediction,
## and can't be augmented with new data. Even worse, they account for 24% of the
## total districts . . .
unspecified_district_ids = set(districts[pd.isna(districts.state)].district_id)
districts = districts[~pd.isna(districts.state)]

for unspecified_district_id in unspecified_district_ids:
    engagement.pop(int(unspecified_district_id))

for district_id, district_dataframe in engagement.items():
    engagement[district_id] = district_dataframe[~pd.isna(district_dataframe.engagement_index)]

products = {row['LP ID'] : dict(row) for _, row in products.iterrows()}

### Create Core Dataset (LearnPlatform)

Essentially, we resolve the district ID and product ID in each row of the engagement data into actual values (so that all the LearnPlatform data is in the same file).

The data for each state -indexed by date and district ID - is saved to its own file.

This process only has to be done once.

In [7]:
def parse_range(x):
    if type(x) == float:
        return x
    elif type(x) == str:
        if '[' in x:
            return AnonymizedNumber(x)
        else:
            return float(x)

In [9]:
for state_name_str in tqdm(fips_state_code_dataframe.STATENAME):
    print(state_name_str)
    
    rows = []
    indices = []
        
    for _, district_row in districts[districts.state.str.lower() == state_name_str.lower()].iterrows():
        district_row = dict(district_row)
        district_id = district_row['district_id']
        
        for _, row in engagement[district_id].iterrows():
            try:
                product_row = products[int(row.lp_id)]
            except KeyError:
                ## It appears the rest of the products were kept in the data despite
                ## not being in the top 372 products? I guess I will assign these to
                ## an "other" category . . .
                product_row = {'Sector(s)': 'Other', 'Primary Essential Function': 'Other'}
            except ValueError:
                print('Invalid Product ID: {} (District ID {}, Day {})'.format(row.lp_id, district_id, row.time))
                continue

            indices.append((datetime.strptime(row.time, '%Y-%m-%d'), district_id))
            rows.append({'engagement;pct_access': row.pct_access,
                         'engagement;engagement_index': row.engagement_index,
                         'products;Sector(s)': numeric_mappings[product_row['Sector(s)']],
                         'products;Primary Essential Function': numeric_mappings[product_row['Primary Essential Function']],
                         'districts;state': district_row['state'],
                         'districts;locale': district_row['locale'],
                         'districts;pct_black/hispanic': parse_range(district_row['pct_black/hispanic']),
                         'districts;pct_free/reduced': parse_range(district_row['pct_free/reduced']),
                         'districts;county_connections_ratio': parse_range(district_row['county_connections_ratio']),
                         'districts;pp_total_raw': parse_range(district_row['pp_total_raw']),
                        })

    if len(rows) > 0:
        index = pd.MultiIndex.from_tuples(indices, names=["time", "district_id"])
        combined_state_dataframe = pd.DataFrame(rows, index=index)
        #combined_state_dataframe.to_csv(Path('.') / 'processed_data' / '{}.csv'.format(state_name_str), index=True)
        combined_state_dataframe.to_pickle(Path('.') / 'processed_data' / '{}.gz'.format(state_name_str), 
                                           compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})
 

  0%|                                                                                           | 0/74 [00:00<?, ?it/s]

ALABAMA
ALASKA
AMERICAN SAMOA
ARIZONA
Invalid Product ID: nan (District ID 9007, Day 2020-12-08)


  5%|████▍                                                                              | 4/74 [00:02<00:37,  1.87it/s]

ARKANSAS
CALIFORNIA
Invalid Product ID: nan (District ID 2074, Day 2020-03-13)
Invalid Product ID: nan (District ID 9357, Day 2020-12-27)
Invalid Product ID: nan (District ID 7164, Day 2020-02-14)
Invalid Product ID: nan (District ID 7164, Day 2020-02-18)
Invalid Product ID: nan (District ID 7164, Day 2020-02-26)
Invalid Product ID: nan (District ID 7164, Day 2020-03-02)
Invalid Product ID: nan (District ID 7164, Day 2020-03-10)
Invalid Product ID: nan (District ID 7164, Day 2020-03-13)
Invalid Product ID: nan (District ID 7164, Day 2020-10-26)
Invalid Product ID: nan (District ID 7164, Day 2020-10-27)


  9%|███████▊                                                                           | 7/74 [01:27<15:32, 13.92s/it]

...

VIRGIN ISLANDS
WASHINGTON
Invalid Product ID: nan (District ID 2567, Day 2020-02-18)
Invalid Product ID: nan (District ID 2567, Day 2020-02-20)
Invalid Product ID: nan (District ID 2567, Day 2020-02-28)
Invalid Product ID: nan (District ID 2567, Day 2020-10-01)
Invalid Product ID: nan (District ID 2130, Day 2020-12-26)


 73%|███████████████████████████████████████████████████████████▊                      | 54/74 [23:12<09:47, 29.36s/it]

WEST VIRGINIA
WISCONSIN
Invalid Product ID: nan (District ID 7752, Day 2020-04-29)
Invalid Product ID: nan (District ID 7752, Day 2020-05-28)


100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [23:29<00:00, 19.04s/it]

WYOMING
AMERICAN SAMOA
FEDERATED STATES OF MICRONESIA
GUAM
JOHNSTON ATOLL
MARSHALL ISLANDS
NORTHERN MARIANA ISLANDS
PALAU
MIDWAY ISLANDS
PUERTO RICO
U.S. MINOR OUTLYING ISLANDS
NAVASSA ISLAND
VIRGIN ISLANDS
WAKE ISLAND
BAKER ISLAND
HOWLAND ISLAND
JARVIS ISLAND
KINGMAN REEF
PALMYRA ATOLL


### Add Other Datasets to the Core Dataset

Parameters should be defined as follows:
    
- ```dataframe```: Pandas dataframe that contains all data to be added to the core dataset
- ```geographic_resolution_str```: What geographic resolution the new data is at (i.e., 'state', 'zipcode')
- ```geographic_id_column_name_str```: What column the identifier for the geographic units of the new data is in
- ```output_columns_dict```:  Dictionary mapping column names in ```dataframe``` to column names as they should appear in the updated core dataset
- ```crosswalk_by```: What type of weight to use when estimating district-level data from non-district level data. Currently only supports 'area', but I am working on a population weight, too. (See the README)

In [32]:
def add_dataset(dataframe, 
                geographic_resolution_str, geographic_id_column_name_str,
                output_columns_dict,
                crosswalk_by='area'):
    
    assert geographic_resolution_str in VALID_SOURCE_GEOGRAPHIC_RESOLUTIONS
    geo_id_col = {'zipcode': 'ZCTA5CE20', 'state': 'STATEFP'}[geographic_resolution_str]
    
    weights_dataframe = pd.read_csv(ALIGNMENT_DATA_DIR_PATH / 'weights' / '{}_weights.csv'.format(geographic_resolution_str))
    weights_dataframe = weights_dataframe.set_index(geo_id_col)

    for state_dataframe_path in tqdm([sdf for sdf in (Path('.') / 'processed_data').iterdir() if sdf.suffix == '.gz']):
        state_dataframe = pd.read_pickle(state_dataframe_path)
        for v in output_columns_dict.values():
            state_dataframe[v] = None

        _tmp = fips_state_code_dataframe[fips_state_code_dataframe.STATENAME == state_dataframe_path.stem].iloc[0]
        state_id = _tmp.STATEFP
        state_name = _tmp.STATENAME.lower()

        for locale_type in weights_dataframe.localetype.unique():
            ## todo: weight non-numeric (categorical) values?
            weighted_output_dict = {v: 0.0 for v in output_columns_dict.values()}
            denominator = 0.0

            statelocale_weights_dataframe = weights_dataframe[(weights_dataframe.STATEFP == state_id) & \
                                                              (weights_dataframe.localetype == locale_type)]

            for geo_id, geo_weights in statelocale_weights_dataframe.iterrows():
                row = broadband[broadband[geographic_id_column_name_str] == geo_id]
                if len(row) == 1:
                    row = row.iloc[0]
                elif len(row) > 1:
                    raise ValueError
                elif len(row) == 0:
                    continue

                for in_col_name, out_col_name in output_columns_dict.items():
                    if not pd.isna(row[in_col_name]):
                        if crosswalk_by == 'area':
                            weighted_output_dict[out_col_name] += (geo_weights.FRACAREA_TARGET * row[in_col_name])
                            denominator += geo_weights.FRACAREA_TARGET
                        elif crosswalk_by == 'population':
                            raise NotImplementedError
                        else:
                            raise NotImplementedError

            if locale_type == 'Suburban':
                locale_type = 'Suburb'
                
            for v, v_val in weighted_output_dict.items():
                state_dataframe.loc[(state_dataframe['districts;state'].str.lower() == state_name) & \
                                    (state_dataframe['districts;locale'] == locale_type), v] = round(v_val / denominator, 5) \
                                                                                               if denominator > 0 else 0.0
        #state_dataframe.to_csv(state_dataframe_path, index=False)
        state_dataframe.to_pickle(state_dataframe_path, 
                                  compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})
    

#### Add BroadbandNow data

In [33]:
broadband = pd.read_csv(RAW_DATA_DIR_PATH / 'BroadbandNow' / 'broadband_data_opendatachallenge.csv', encoding='ANSI')

def parse(x):
    if pd.isna(x):
        return None
    elif x.endswith('%'):
        return float(x[:-1]) / 100.0
    else:
        return round(float(x), 2)
    
broadband['%Access to Terrestrial Broadband'] = broadband['%Access to Terrestrial Broadband'].apply(parse)

add_dataset(broadband,
            'zipcode', 'Zip',
            crosswalk_by='area',
            output_columns_dict={'Wired25_3_2020': 'broadband;wired_over_25',
                                 'AverageMbps': 'broadband;avg_mbps',
                                 '%Access to Terrestrial Broadband': 'broadband;frac_access',
                                 'Lowest Priced Terrestrial Broadband Plan': 'broadband;lowest_price',
                                })

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [01:19<00:00,  3.45s/it]
